In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [2]:
train_address="/kaggle/input/titanic/train.csv"
test_address="/kaggle/input/titanic/test.csv"
train_set=pd.read_csv(train_address)
test_set=pd.read_csv(test_address)
y=train_set.Survived
Id=test_set.PassengerId

In [3]:
def cont(x):
    if 'Mr' in x:
        return 'Mr'
    elif 'Mrs' in x:
        return 'Mrs'
    elif 'Miss' in x:
        return 'Miss'
    else:
        return 'other'
    
train_set['Name']=train_set['Name'].apply(lambda x:cont(x))
test_set['Name']=test_set['Name'].apply(lambda x:cont(x))
uniq={col:len(train_set[col].unique()) for col in train_set.columns }
missing={col:train_set[col].isnull().sum() for col in train_set.columns}

In [4]:
%matplotlib notebook
import matplotlib.pyplot as plt
plt.figure()
plt.bar(uniq.keys(),uniq.values(),width=0.3,label='unique')
plt.bar(missing.keys(),missing.values(),width=0.3,label='missing')
plt.legend()
plt.xticks(rotation=90)
#No the basis of the graph we remove the some columns which are given in the next cell

<IPython.core.display.Javascript object>

([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 <a list of 12 Text major ticklabel objects>)

In [5]:
y=train_set.Survived
Id=test_set.PassengerId
#pos={train_set.loc[i,'Pclass']:train_set.loc[i,'Sex'] for i in train_set.index if (train_set.loc[i,'Survived']==0)}
X=train_set.drop({'PassengerId','Ticket','Cabin','Survived'},axis=1)
test=test_set[X.columns]
print(X.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Name      891 non-null    object 
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(3)
memory usage: 55.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Name      418 non-null    object 
 2   Sex       418 non-null    object 
 3   Age       332 non-null    float64
 4   SibSp     418 non-null    int64  
 5   Parch     418 non-null    int64  
 6   Fare      417 non

## Data Leakage
To avoid data leakage we will first split data into training and validation and then fill out the missing values in it.

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=1)
X_train['Age'].fillna(X_train['Age'].mean(),inplace=True)
X_train['Embarked'].fillna(X_train['Embarked'].mode()[0],inplace=True)
X_test['Age'].fillna(X_test['Age'].mean(),inplace=True)
X_test['Embarked'].fillna(X_test['Embarked'].mode()[0],inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [7]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
cat_col=[col for col in X_train.columns if X_train[col].dtype=='object']
for col in cat_col:
    X_train[col]=encoder.fit_transform(X_train[col])
    X_test[col]=encoder.transform(X_test[col])
X_train.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
35,1,1,1,42.000000,1,0,52.0000,2
46,3,1,1,30.073682,1,0,15.5000,1
453,1,1,1,49.000000,1,0,89.1042,0
291,1,1,0,19.000000,1,0,91.0792,0
748,1,1,1,19.000000,1,0,53.1000,2


# Normalization
We can the range of some feature vary in large size than others like 'Sibsp' and 'Parch' may have same order but not in the range of 'Age' and 'Fare'.We will put all of them in the same order between 0-1.


In [8]:
column=X_train.columns
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(X_train)
X_trainS=pd.DataFrame(scaler.transform(X_train),columns=column)
X_testS=pd.DataFrame(scaler.transform(X_test),columns=column)
X_trainS.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,0.0,0.5,1.0,0.522493,0.125,0.0,0.101497,1.0
1,1.0,0.5,1.0,0.372627,0.125,0.0,0.030254,0.5
2,0.0,0.5,1.0,0.610455,0.125,0.0,0.173920,0.0
3,0.0,0.5,0.0,0.233476,0.125,0.0,0.177775,0.0
4,0.0,0.5,1.0,0.233476,0.125,0.0,0.103644,1.0


y_train.index=X_trainS.index
X_trainS['Sur']=y_train
X_trainS.head()

import seaborn as sns
sns.pairplot(X_trainS,dropna=True,hue='Sur',diag_kws={'bw': 0.2})

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
model1=KNeighborsClassifier()
grid_val1={'n_neighbors':range(1,50,4)}
grid1=GridSearchCV(model1,param_grid=grid_val1,scoring='accuracy')
grid1.fit(X_trainS,y_train)
print(grid1.best_params_)
print(accuracy_score(y_test,grid1.predict(X_testS)))

{'n_neighbors': 17}
0.7892376681614349


In [10]:
from sklearn.linear_model import LogisticRegression
model2=LogisticRegression()
grid_val2={'C':[0.0001,0.001,0.01,0.1,1,3,6,10,12,15]}
grid2=GridSearchCV(model2,param_grid=grid_val2,scoring='accuracy')
grid2.fit(X_trainS,y_train)
print(grid2.best_params_)
print(accuracy_score(y_test,grid2.predict(X_testS)))

{'C': 3}
0.7982062780269058


In [11]:
from sklearn.svm import SVC
model3=SVC(kernel='rbf')
grid_val3={'gamma':[0.0001,0.001,0.01,0.1,1,3,6,10,12,15],'C':[0.0001,0.001,0.01,0.1,1,3,6,10,12,15]}
grid3=GridSearchCV(model3,param_grid=grid_val3,scoring='accuracy')
grid3.fit(X_trainS,y_train)
print(grid3.best_params_)
print(accuracy_score(y_test,grid3.predict(X_testS)))

{'C': 1, 'gamma': 1}
0.7847533632286996


In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Name      418 non-null    object 
 2   Sex       418 non-null    object 
 3   Age       332 non-null    float64
 4   SibSp     418 non-null    int64  
 5   Parch     418 non-null    int64  
 6   Fare      417 non-null    float64
 7   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(3)
memory usage: 26.2+ KB


In [13]:
test['Age'].fillna(test['Age'].mean(),inplace=True)
test['Fare'].fillna(test['Fare'].mean(),inplace=True)
for col in cat_col:
    test[col]=encoder.fit_transform(test[col])
test_S=pd.DataFrame(scaler.fit_transform(test),columns=column)
test_S.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1.0,0.5,1.0,0.452723,0.000,0.000000,0.015282,0.5
1,1.0,0.5,0.0,0.617566,0.125,0.000000,0.013663,1.0
2,0.5,0.5,1.0,0.815377,0.000,0.000000,0.018909,0.5
3,1.0,0.5,1.0,0.353818,0.000,0.000000,0.016908,1.0
4,1.0,0.5,0.0,0.287881,0.125,0.111111,0.023984,1.0


In [14]:
predictions = grid2.predict(test_S)

output = pd.DataFrame({'PassengerId':Id, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)